## Read in the data in wide format

In [14]:
from Datapreprocessing import DataProcessor
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
train_folder = 'train_tsv1'
test_folder = 'test_tsv1'
train_metadata_path = 'metadata/training_metadata.csv'
test_metadata_path = 'metadata/test_metadata.csv'

In [16]:
processor = DataProcessor(train_folder, test_folder, train_metadata_path, test_metadata_path)
train_data, test_data = processor.prepare_datasets(wide_format=True)

In [17]:
train_data.head()

,participant_id,corr_1,corr_2,corr_3,corr_4,corr_5,corr_6,corr_7,corr_8,corr_9,...,bmi,ethnicity,race,handedness,parent_1_education,parent_2_education,p_factor_fs,internalizing_fs,externalizing_fs,attention_fs
0,NDAREE015WGU,0.074628,0.673757,0.813541,0.725489,0.823861,0.801859,0.690204,0.450536,0.602295,...,19.10,Not_Hispanic_or_Latino,Other,Right,Complete_tertiary,Complete_tertiary,0.59,-0.15,0.29,-1.60
1,NDARNV694EMG,0.167241,0.549381,0.453434,0.132590,0.496156,0.337475,0.506641,0.215110,0.564849,...,32.46,Not_Hispanic_or_Latino,White,Right,Complete_primary,Complete_tertiary,-0.17,0.54,-1.00,-0.87
2,NDARFR108JNB,0.197198,0.337779,0.018934,0.580793,0.728406,0.507803,0.470236,0.401553,0.547191,...,15.98,Hispanic_or_Latino,Other,Right,Complete_tertiary,Complete_tertiary,-0.43,-0.25,0.18,0.39
3,NDARZF170TZ0,0.370992,0.546226,0.659874,0.212565,0.429793,0.352411,0.047157,0.112740,0.012267,...,14.92,Not_Hispanic_or_Latino,Other,Right,Complete_primary,Complete_tertiary,-0.33,0.25,-0.31,1.10
4,NDARYJ334EHH,0.127956,0.415074,0.219740,0.311787,0.650690,0.615524,0.597610,0.334451,0.327396,...,18.85,Hispanic_or_Latino,NaN,Right,NaN,No_incomplete primary,-1.04,-0.40,-0.53,0.71


In [18]:
train_data.to_csv('train_data_wide.csv', index=False)
test_data.to_csv('test_data_wide.csv', index=False)

## Fill in the missing values

In [7]:
# Replace missing BMI values with 0
train_data['bmi'] = train_data['bmi'].fillna(0)

# Replace missing values in categorical columns with "not recorded"
categorical_columns = ['ethnicity', 'race', 'parent_1_education', 'parent_2_education']
train_data[categorical_columns] = train_data[categorical_columns].fillna("not recorded")

In [8]:
from sklearn.feature_selection import SelectKBest, f_regression

object_columns = train_data.select_dtypes(include=['object']).columns
correlation_features = train_data.drop(columns=object_columns)
target = train_data['age']
selector = SelectKBest(f_regression, k=100)
features_reduced = selector.fit_transform(correlation_features, target)

In [13]:
features_reduced = pd.DataFrame(features_reduced, columns=correlation_features.columns[selector.get_support()])
features_reduced.head()

,corr_4440,corr_6700,corr_6701,corr_6805,corr_7055,corr_7062,corr_7070,corr_7231,corr_7278,corr_7279,...,corr_18761,corr_18763,corr_18869,corr_18897,corr_19391,corr_19392,corr_19422,corr_19796,age,bmi
0,-0.121599,0.155337,0.141229,0.124573,-0.368117,-0.199834,-0.295317,-0.248700,0.258086,0.154537,...,-0.292097,-0.367697,-0.193657,-0.100102,-0.268719,-0.184924,0.133518,0.232407,13.35,19.10
1,-0.047480,0.108901,0.096683,0.117455,-0.125318,-0.032815,-0.298915,0.042610,-0.068114,0.017172,...,-0.156705,-0.422065,-0.207533,0.052250,-0.402520,-0.228050,-0.153885,0.235959,14.62,32.46
2,-0.009281,0.067010,0.032118,0.020547,-0.469052,-0.148011,0.215291,0.111302,-0.051266,0.027710,...,-0.207746,-0.151859,-0.435711,0.092661,0.063883,0.120454,0.051956,0.036671,7.14,15.98
3,0.331472,0.309834,0.122446,0.267139,-0.293554,-0.205999,-0.333704,-0.088601,-0.020188,0.279073,...,-0.023780,-0.337322,-0.097995,0.278202,-0.117643,0.211957,-0.095803,0.323449,13.94,14.92
4,0.252408,0.072247,-0.041780,0.261419,-0.206515,-0.449440,-0.673377,0.132281,-0.273004,0.367468,...,0.091789,-0.238256,0.058237,0.191942,-0.291110,-0.056419,0.138968,0.419641,9.92,18.85
